In [3]:
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
APIFY_TOKEN = os.getenv("APIFY_API_KEY")

# Ensure the token is present
if not APIFY_TOKEN:
    raise OSError("APIFY_API_KEY not found in environment variables.")

# TikTok video URL to scrape
tiktok_url = "https://www.tiktok.com/@cioccatalin1/video/7456347160892706070"

# API endpoint
run_url = f"https://api.apify.com/v2/acts/clockworks~tiktok-comments-scraper/run-sync-get-dataset-items?token={APIFY_TOKEN}"

# Input payload for the actor
payload = {
    "postURLs": [tiktok_url],
    "commentsPerPost": 50,
    "maxRepliesPerComment": 0,
    "resultsPerPage": 50,
}

# Make the POST request
response = requests.post(run_url, json=payload)
response.raise_for_status()
comments = response.json()

# Print a few comments
for i, comment in enumerate(comments[:5], start=1):
    print(f"{i}. {comment.get('text')} (by {comment.get('username')})")

1. La Ferrari più bella che ci sia!! (by None)
2. ma quanto è bella sempre intramontabile 💪 (by None)
3. ma davvero c'e' chi preferisce la countach? (by None)
4. Le supersportive Italiane,sono le migliori in assoluto....... (by None)
5. Bellezza e fascino senza tempo😍😍😍, la Ferrari testa rossa ha linee belle. che sembra uscita ora dalla fabbrica (by None)
